In [1]:
import scipy.stats as sts
import dynesty
import time
from dynesty import plotting as dyplot
from numpy import *
import ipyparallel as ipp

class Pool(object):

    def __init__(self, dview, nprocs):
        self.dview = dview
        self.size = nprocs

    def map(self, function, tasks):
        return self.dview.map_sync(function, tasks)

### FUNCTIONS
import colorcet as cc
from matplotlib.widgets import Slider, Button
import pandas
from scipy.linalg import expm, norm
import scipy.constants as sc
import math
import time
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from numpy import cross, eye, dot
import numpy as np
import traceback
%matplotlib widget


# Three variables drawn from Gaussian distribution

def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec

# Doppler shift a given angular frequency given angle to observer in source frame and observer's speed


def doppler_shift(angFreq, obsAngle, obsSpeed):
    # obsAngFreq = angFreq*((np.sqrt(1-obsSpeed**2)/(1 - obsSpeed*np.cos(obsAngle)))) #old function—used wrong equation
    obsAngFreq = angFreq*((obsSpeed*np.cos(obsAngle)+1)/np.sqrt(1-obsSpeed**2))
    return obsAngFreq


def ang_freq_to_lambda(omega):
    wavelength = sc.c/(2*math.pi*omega)
    return wavelength


def lambda_to_ang_freq(wavelength):
    omega = sc.c/(2*math.pi*wavelength)
    return omega


def vectors_angle(v1, v2):
    if (np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)) <= 1 and (np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)) >= -1:
        return math.acos((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    elif (np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)) < -1:
        print('Less:' + str((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))))
        return math.acos(-1)
    elif (np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)) > 1:
        print('Greater:' + str((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))))
        return math.acos(1)
    
    # try:
    #     angle = math.acos((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    #     # print((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    #     return angle
    # except ValueError:
    #     print((np.dot(v1, v2))/(np.linalg.norm(v1)*np.linalg.norm(v2)))
    #     quit()


def abberation_angle(angle, obsSpeed):
    abberationAngle = math.acos(
        (obsSpeed + math.cos(angle))/(obsSpeed*math.cos(angle)+1))
    return abberationAngle


def unit_norm_vector(v1, v2):
    normVector = np.cross(v1, v2)
    unitNormVector = normVector/np.linalg.norm(normVector)
    return unitNormVector


def rotateMatrix(axis, angle):
    rotatedMatrix = expm(cross(eye(3), axis/norm(axis)*angle))
    return rotatedMatrix


def wavelength_to_rgb(wavelength, gamma=0.8):
    '''This converts a given wavelength of light to an 
    approximate RGB color value. The wavelength must be given
    in nanometers in the range from 380 nm through 750 nm
    (789 THz through 400 THz).

    Based on code by Dan Bruton
    http://www.physics.sfasu.edu/astro/color/spectra.html
    '''

    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    R *= 255
    G *= 255
    B *= 255
    return [int(R), int(G), int(B)]


def determineAngleToObs(obsVector, xi, yi, zi):
    # angles between all vectors and observer vector from dot product
    angles = []
    for i in range(0, len(xi)):
        angles.append(vectors_angle(obsVector, (xi[i], yi[i], zi[i])))
    return angles


def restLambdaToObsLambda(restLambda, angles, obsSpeed):
    # lambda values as perceived by moving observer
    restAngFreq = lambda_to_ang_freq(restLambda)
    obsLambda = (ang_freq_to_lambda(
        doppler_shift(restAngFreq, angles, obsSpeed)))
    return obsLambda


def wavelengthToRGB(obsLambda):
    RGBs = []
    for i in range(0, len(obsLambda)):
        RGBs.append(wavelength_to_rgb(obsLambda[i]))
    RGBs = np.asarray(RGBs, dtype=np.float32)
    return RGBs


def updatePoints(angles, obsSpeed, obsVector, xi, yi, zi):
    ## Now we try to congregate points along the source of motion due to abberation
    # angle difference
    deltaAngles = []
    for i in range(0, len(angles)):
        deltaAngles.append(
            -(abberation_angle(angles[i], obsSpeed) - angles[i]))
    # normal vector/rotation axes
    rotAxes = []
    for i in range(0, len(angles)):
        rotAxes.append(unit_norm_vector((xi[i], yi[i], zi[i]), obsVector))
    rotationMatrices = []
    for i in range(0, len(rotAxes)):
        rotationMatrices.append(rotateMatrix(rotAxes[i], deltaAngles[i]))
    rotatedPoints = []
    for i in range(0, len(rotationMatrices)):
        rotatedPoints.append(dot(rotationMatrices[i], (xi[i], yi[i], zi[i])))
    rotatedPoints = np.asarray(rotatedPoints, dtype=np.float32)
    return rotatedPoints


def transformedPoints(obsSpeed, obsVector, restLambda, xi, yi, zi):
    angles = determineAngleToObs(obsVector, xi, yi, zi)
    obsLambda = restLambdaToObsLambda(restLambda, angles, obsSpeed)*10**9
    RGBs = wavelengthToRGB(obsLambda)
    rotatedPoints = updatePoints(angles, obsSpeed, obsVector, xi, yi, zi)
    return rotatedPoints, RGBs, obsLambda


def cart2sph(obsVector):
    hxy = np.hypot(obsVector[0], obsVector[1])
    if obsVector[2] == 0:
        theta = np.pi/2
    else:
        theta = np.arccos(
            obsVector[2]/math.sqrt(obsVector[0]**2 + obsVector[1]**2 + obsVector[2]**2))
    if obsVector[0] > 0:
        atan = np.arctan(obsVector[1]/obsVector[0])
        phi = atan
    elif obsVector[0] < 0 and obsVector[1] >= 0:
        atan = np.arctan(obsVector[1]/obsVector[0])
        phi = atan + np.pi
    elif obsVector[0] < 0 and obsVector[1] < 0:
        atan = np.arctan(obsVector[1]/obsVector[0])
        phi = atan - np.pi
    elif obsVector[0] == 0 and obsVector[1] > 0:
        phi = np.pi/2
    elif obsVector[0] == 0 and obsVector[1] < 0:
        phi = -np.pi/2
    elif obsVector[0] == 0 and obsVector[1] == 0:
        phi = 0
    return (theta, phi)


def sph2cart(obsPolar):  # r=1
    x = np.sin(obsPolar[0]) * np.cos(obsPolar[1])
    y = np.sin(obsPolar[0]) * np.sin(obsPolar[1])
    z = np.cos(obsPolar[0])
    return (x, y, z)


def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

################### DATA

sigma = 0.01
restLambda = ang_freq_to_lambda(1)  # define ang_freq to be 1.
obsPolar = (0.7, 4)
# time_taken = []
# pol_true, az_true = obsPolar
# logZ = []
# log_Z0 = []
# log_ZCMB = []
omega_val = 1
n = 200 # number of points to sample
obsSpeed = 0.001

xi, yi, zi = sample_spherical(n)
rotatedPoints, RGBs, obsLambda = transformedPoints(
    obsSpeed, sph2cart(obsPolar), restLambda, xi, yi, zi)

# list of Doppler shifted angular frequencies
obsAngFreq = lambda_to_ang_freq(obsLambda*10**(-9))
normalisedObsAngFreq = NormalizeData(obsAngFreq)

## dynesty model fitting
# convert obsLambda in nm to m, then to angular frequency
omegas = lambda_to_ang_freq(obsLambda*10**(-9))
omega_error = np.random.normal(loc=0, scale=sigma, size=len(omegas))
omega_new = omegas + omega_error

#################### BAYESIAN FUNCTIONS

def model(v, az, pol):
    observerVector = sph2cart((pol, az))
    alphaDash = determineAngleToObs(observerVector, rotatedPoints[0:len(
        rotatedPoints), 0], rotatedPoints[0:len(rotatedPoints), 1], rotatedPoints[0:len(rotatedPoints), 2])
    model = lambda_to_ang_freq(restLambda) * \
        (np.sqrt(1-v**2)/(1-v*np.cos(alphaDash)))
    return model

def prior_transform(uTheta):
    uV, uAz, uPol = uTheta
    v = 0.01*uV # uniform between 0 and 0.01
    az = 2*np.pi*uAz # uniform between 0 and 2*pi
    pol = np.pi*uPol # uniform between 0 and pi
    return v, az, pol

def lnlike(Theta):
    v, az, pol = Theta
    # turn az and pol into alpha' for this omega
    omega_val = model(v,az,pol)
    return sum(sts.norm.logpdf(omega_new,loc=omega_val,scale=sigma))

############ Parallel

ndim = 3

#Starting Client
cl = ipp.Cluster(n=4)
rc = cl.start_and_connect_sync()

# rc = ipp.Client()
nprocs = len(rc.ids)
print(rc.ids)

dview = rc[:]
dview.use_dill()

# dview['sampling_parameters'] = sampling_parameters

#sync imports on all engines
with dview.sync_imports():
    import numpy
    import scipy
    from scipy import stats
    import dynesty

dview.push(dict(sigma=sigma,restLambda=restLambda,obsPolar=obsPolar,omega_val=omega_val,n=n,obsSpeed=obsSpeed,rotatedPoints=rotatedPoints,omega_new=omega_new))

pool = Pool(dview, nprocs)

dsampler = dynesty.NestedSampler(lnlike, prior_transform,
                                 ndim, pool=pool)

dsampler.run_nested()


Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

NameError: name 'parameters' is not defined